In [1]:
import os
import cv2
drawing = False
temp_folder = "./temp"
video_path = './Neuer_Moments.mp4'

cap = cv2.VideoCapture(video_path)

def mouse_callback(event, x, y, flags, param):
    global x1, y1, x2, y2, drawing
    if event == cv2.EVENT_LBUTTONDOWN:
        x1, y1 = x, y
        x2, y2 = x, y
        drawing = True
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            x2, y2 = x, y
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False

скрипт для вибору об'єкта вручну
використання: 
- нажимати любу кнопку щоб пропустити кадр.
- коли знайшли потрібний кадр - виділити курсором об'єкт (немає ніяких візуальних підказок що ви виділили обєкт)
- після виділення об'єкта - нажміть s
В директорії створиться зображення з назвою first_frame.jpg на якому зможете переглянути виділення.

In [2]:
cv2.namedWindow('Frame')
cv2.setMouseCallback('Frame', mouse_callback)
x1, y1 = 0, 0
x2, y2 = 0, 0

# Ітерувати по кадрах відео
ret, frame = cap.read()
while ret:
    # Відобразити кадр
    cv2.imshow('Frame', frame)
    
    # Зупинитись коли нажата клавіша s
    key = cv2.waitKey(0)
    if key == ord('s'):
        first_frame = frame.copy()
        cv2.rectangle(first_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.imwrite('first_frame.jpg', first_frame)
        break
    
    # Читати наступний кадр
    ret, frame = cap.read()

# Зберегти перший кадр з об'єктом
cv2.destroyAllWindows()

формую координати.
створюю відстежувач за координатами на вибраному першому фото

In [3]:
width, height = x2-x1, y2-y1
bbox = (x1, y1, width, height)

tracker = cv2.TrackerKCF_create()
# tracker = cv2.TrackerMIL_create()
# tracker = cv2.TrackerCSRT_create()

ok = tracker.init(first_frame, bbox)

скрипт для проходження по всім кадрам відео, та малювання рамок з допомогою трекера

In [4]:
if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)
frame_count = 0
cap = cv2.VideoCapture(video_path)
while True:
    # Читати наступний кадр
    ret, frame = cap.read()
    if not ret:
        break
    
    # Оновити відстежувач
    ok, bbox = tracker.update(frame)
    
    # Накласти прямокутник на кадр
    if ok:
        x, y, w, h = [int(v) for v in bbox]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Відобразити кадр
    cv2.imshow('Tracking', frame)
    
    # Зберегти кадр
    frame_path = temp_folder + f'/frame_{frame_count}.jpg'
    cv2.imwrite(frame_path, frame)
    frame_count += 1
    
    # Затримка для відображення
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
cv2.destroyAllWindows()